# Using Regular Expressions in Python 3

Este notebook é uma adaptação da [segunda parte](https://regexone.com/references/python) do [curso sobre expressões regulares]([RegexOne](https://regexone.com/lesson) do site [RegexOne](https://regexone.com).

Python dá suporte a expressões regulares através da biblioteca padrão ```re```, disponibilizada em todas as instalações Python. Note que diferentes linguagens apresentam sintaxes variadas para expressões regulares. Aqui, vamos usar a sintaxe do Python, que tenta seguir o padrão [PCRE](https://www.pcre.org/).

## Strings cruas do Python (raw strings)

Ao escrever expressões regulares em Python, é recomendável usar *raw strings* no lugar das strings tradicionais do Python (```str```). *Raw strings* começam com o prefixo especial **r**, que indica ao interpretador Python que ignore caracteres especiais.

Na prática, isto significa que uma *raw string* como ```r"\n"``` pode ser usada para buscar por uma quebra de linha. Usando strings normais, seria necessário o uso de caracteres de escape para que o caracter ```\n``` fosse interpretado corretamente (```"\\n"```), tornando as expressões regulares menos legíveis.

In [1]:
a = "\n"
print("Teste:", a)
b = r"\n"
print("Teste:", b)
c = "\\n"
print("Teste:", c)

Teste: 

Teste: \n
Teste: \n


## Casando uma string

O pacote ```re``` disponibiliza vários procedimentos. Para testar se uma expressão regular casa com uma string (**match**), podemos usar os procedimentos ```search()```, ```match()``` ou ```fullmatch()```:

* **search**: Percorre a string para ver se alguma de suas substrings casa com a expressão passada.
```python
result = search(pattern, input_str, flags=0)
```
* **match**: Tenta casar a expressão passada com o início da string.
```python
result = match(pattern, input_str, flags=0)
```
* **fullmatch**: Tenta casar a expressão passada com a string inteira.
```python
result = fullmatch(pattern, input_str, flags=0)
```

Seja qual for o procedimento usado, o casamento bem sucedido da expressão retorna um objeto MatchObject, que contém informações sobre a (sub)string casada. Caso não haja casamento, a referência ```None``` é retornada.

Note que mesmo o procedimento ```search()``` para de percorrer a string quando um casamento ocorre. Assim, esses métodos devem ser usados para verificar se uma string corresponde a uma expressão regular e não para extrair substrings de um texto.

### Exemplo

### Exemplo

Vamos usar uma expressão regular para casar uma data:

In [2]:
from re import search

regex = r"(\d+) de ([a-zA-Z]+)"
if search(regex, "Hoje é 24 de junho"):
    print("O padrão regex casa com a string informada! :)")
else:
    print("O padrão regex não casa com a string informada. :(")

O padrão regex casa com a string informada! :)


Também é possível usar os métodos do objeto ```MatchObject``` retornado para mais informações:
- os métodos ```start()``` e ```end()``` informam o início e o fim da primeira substring que casa com a expressão informada.
- o método ```group()``` retorna o casamento inteiro e os grupos capturados.

In [3]:
result = search(regex, "Hoje é 24 de junho")
print("Casamento da substring [%s, %s)" % (result.start(), result.end()))

Casamento da substring [7, 18)


Os grupos capturados são acessados a partir do índice 1. O índice 0 se refere ao casamento inteiro. 
* ```result.group(0)``` sempre retorna a substring inteira.
* ```result.group(1), result.group(2), ...``` retornam os grupos capturados, contados da esquerda para a direita.
* ```result.group()``` é equivalente a ```result.group(0)```

In [4]:
print("Casamento: %s" % (result.group(0)))
print("Dia: %s" % (result.group(1)))
print("Mês: %s" % (result.group(2)))

Casamento: 24 de junho
Dia: 24
Mês: junho


## Capturando grupos

Para extrair informações de um texto, podemos usar os procedimentos ```findall()``` ou ```finditer()```:

* **findall**: Quando a expressão apresenta grupos, retorna uma lista contendo todos os dados capturados. Caso contrário, retorna uma lista contendo todos os casamentos efetuados.
```python
matchList = findall(pattern, input_str, flags=0)
```
* **finditer**: Idêntico a findall(), porém retorna uma lista de referências para objetos MatchObject. 
```python
matchList = re.finditer(pattern, input_str, flags=0)
```

### Exemplo

Vamos usar uma expressão regular para casar algumas datas:}

In [5]:
from re import findall, finditer

regex = r"\d+ de [a-zA-Z]+"
matches = findall(regex, "Minha mãe nasceu em 09 de abril. Eu nasci em 23 de fevereiro. Minha filha nasceu em 01 de setembro.")
for match in matches:
    print("Casamento: %s" % (match))    

Casamento: 09 de abril
Casamento: 23 de fevereiro
Casamento: 01 de setembro


Para capturar o mês específico de cada data, podemos usar um grupo:

In [6]:
regex = r"\d+ de ([a-zA-Z]+)"
matches = findall(regex, "Minha mãe nasceu em 09 de abril. Eu nasci em 23 de fevereiro. Minha filha nasceu em 01 de setembro.")
for match in matches:
    print("Casamento: %s" % (match))

Casamento: abril
Casamento: fevereiro
Casamento: setembro


Se precisamors de mais informações sobre cada casamento (ou grupo), usamos ```finditer()```:

In [7]:
regex = r"\d+ de ([a-zA-Z]+)"
matches = finditer(regex, "Minha mãe nasceu em 09 de abril. Eu nasci em 23 de fevereiro. Minha filha nasceu em 01 de setembro.")
for match in matches:
    print("Casamento: [%s, %s)" % (match.start(), match.end()))

Casamento: [20, 31)
Casamento: [45, 60)
Casamento: [84, 98)


## Buscando e substituindo

Outra tarefa comum é buscar e substituir uma parte de uma string usando expressões regulares. 

Um exemplo comum é alterar um delimitador usado em um arquivo de dados tabulados.

Em Python, podemos usar o procedimento ```sub()``` para isso:
```python
replacedString = sub(pattern, replacement_pattern, input_str, count, flags=0)
```

O argumento opcional ```count``` determina o número máximo de substituições a serem feitas. Caso seu valor seja nulo ou negativo, todas as ocorrências encontradas são substituídas.

### Exemplo

Vamos tentar inverter a ordem do dia e do mês na frase anterior. Note que o padrão de substituição também é escrito como uma *raw_string*:

In [8]:
from re import sub

regex = r"(\d+) de ([a-zA-Z]+)"
ordem = sub(regex, r"\2, no dia \1", "Minha mãe nasceu em 09 de abril. Eu nasci em 23 de fevereiro. Minha filha nasceu em 01 de setembro.")
print(ordem)

Minha mãe nasceu em abril, no dia 09. Eu nasci em fevereiro, no dia 23. Minha filha nasceu em setembro, no dia 01.


In [9]:
print(sub(r"\.\s", r".\n", ordem))

Minha mãe nasceu em abril, no dia 09.
Eu nasci em fevereiro, no dia 23.
Minha filha nasceu em setembro, no dia 01.


## Flags

Todos os procedimentos do módulo ```re``` vistos acima aceitam flags como parâmetros opcionais.

A maior parte das flags disponíveis podem ser especificadas diretamente nas expressões regulares, mas em alguns casos pode ser útil utilizá-las como parâmetros:

* ```IGNORECASE``` faz o padrão se tornar insensível a caso.
* ```MULTILINE``` é necessário quando a string apresenta quebras de linha (```\n```). Com esta flag, os metacaracteres de início (```^```) e fim (```$```) passam a ser analisados por linha.
* ```DOTALL``` faz com que o metacaracter ```.``` case com qualquer caracter, incluindo a quebra de linha (```\n```).

## Compilando expressões para ter performance

Criar uma nova expressão regular em Python cada vez que se deseja casar strings é uma abordagem pouco eficiente.

Nesse caso, é recomendado que se compile a expressão regular antes de processar as strings alvo:

```regexObject = compile(pattern, flags=0)```

Este procedimento cria um objeto do tipo RegexObject.

Esta classe implementa como métodos todos os procedimentos descritos neste notebook, mas você não precisa mais passar sua expressão regular como parâmetro a cada chamada (nem eventuais flags).

### Exemplo

Vamos criar uma expressão e extrair informações:

In [10]:
from re import compile
regex = compile(r"(\w+) World")
result = regex.search("Hello World é o mais fácil")
if result:
    print(result.start(), result.end())

0 11


Usando o mesmo objeto, vamos imprimir todos os grupos capturados:

In [11]:
for result in regex.findall("Hello World, Ciao World"):
    print(result)

Hello
Ciao


Por último, vamos substituir Mundo por Terra:

In [12]:
print(regex.sub(r"\1 Earth", "Hello World"))

Hello Earth
